In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path, output_txt_path="pq_corpus.txt"):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(text)

# Example usage
extract_text_from_pdf("pq_textbook.pdf")


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import pipeline


# Load the tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")

# Load the plain text dataset
datasets = load_dataset('text', data_files={'train': 'pq_corpus.txt'})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

# Set up data collator for MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./distilbert-pq-mlm",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

# Run training
trainer.train()

model_save_path = "./distilbert-pq-mlm"

# Save the domain-adapted model
trainer.save_model("./distilbert-pq-mlm")
tokenizer.save_pretrained(model_save_path)

# Verify saved files
print(f"Model and tokenizer saved to: {model_save_path}")

C:\Users\rmathew2020\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generating train split: 26838 examples [00:00, 797084.97 examples/s]
 10%|▉         | 500/5034 [30:14<5:32:29,  4.40s/it]

{'loss': 3.1743, 'grad_norm': 22.645418167114258, 'learning_rate': 4.503377036154152e-05, 'epoch': 0.3}


 20%|█▉        | 1000/5034 [59:01<3:56:10,  3.51s/it]

{'loss': 2.8826, 'grad_norm': 18.324779510498047, 'learning_rate': 4.0067540723083036e-05, 'epoch': 0.6}


...

{'loss': 2.2569, 'grad_norm': 16.32189178466797, 'learning_rate': 5.30393325387366e-06, 'epoch': 2.68}


 99%|█████████▉| 5000/5034 [4:38:24<01:44,  3.06s/it]

{'loss': 2.2809, 'grad_norm': 18.75294303894043, 'learning_rate': 3.3770361541517685e-07, 'epoch': 2.98}


100%|██████████| 5034/5034 [4:40:05<00:00,  3.34s/it]


{'train_runtime': 16805.8714, 'train_samples_per_second': 4.791, 'train_steps_per_second': 0.3, 'train_loss': 2.577739136627282, 'epoch': 3.0}


281 mins 40.6 sec

In [9]:

# Test loading
test_tokenizer = AutoTokenizer.from_pretrained(model_save_path)
test_model = AutoModelForMaskedLM.from_pretrained(model_save_path)

# Create fill-mask pipeline
fill_mask = pipeline("fill-mask", model=test_model, tokenizer=test_tokenizer)

# Example test sentences relevant to power quality
test_sentences = [
    "A sudden [MASK] in voltage can cause sensitive equipment to fail.",
    "The power [MASK] was caused by lightning strike.",
    "The distribution [MASK] failed due to overload.",
    "Multiple [MASK] faults occurred during the thunderstorm.",
    "The [MASK] waveform showed significant distortion.", 
    "[MASK] fell on power lines during a storm, affecting multiple feeders; estimated restoration time is 4 hours."
]

# Run predictions
print("\nTesting domain-adapted model:")
print("-" * 50)
for sentence in test_sentences:
    print(f"\nInput: {sentence}")
    results = fill_mask(sentence)
    print("Top predictions:")
    for r in results[:5]:  # Show top 5 predictions
        print(f"- {r['token_str']}: {r['score']:.4f}")

Device set to use cpu



Testing domain-adapted model:
--------------------------------------------------

Input: A sudden [MASK] in voltage can cause sensitive equipment to fail.
Top predictions:
- increase: 0.4798
- drop: 0.1674
- rise: 0.1308
- change: 0.0904
- surge: 0.0441

Input: The power [MASK] was caused by lightning strike.
Top predictions:
- problem: 0.1721
- loss: 0.1222
- quality: 0.0741
- interruption: 0.0732
- system: 0.0654

Input: The distribution [MASK] failed due to overload.
Top predictions:
- system: 0.5795
- feeder: 0.1961
- line: 0.0223
- systems: 0.0182
- lines: 0.0142

Input: Multiple [MASK] faults occurred during the thunderstorm.
Top predictions:
- ground: 0.1447
- surge: 0.0926
- lightning: 0.0635
- fault: 0.0274
- transient: 0.0264

Input: The [MASK] waveform showed significant distortion.
Top predictions:
- first: 0.1249
- resulting: 0.0728
- initial: 0.0675
- input: 0.0616
- original: 0.0550

Input: [MASK] fell on power lines during a storm, affecting multiple feeders; estimated